In [1]:
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sctools import integrate, plot, io

Global seed set to 0


## Cell abundance analysis with Milo

In [2]:
# load data
adatas = {}
file_names = {
    'tcells.tissue.scps': '../data/tcells.tissue.scps.integrated.h5ad',
    'tcells.tissue.ad': '../data/tcells.tissue.ad.integrated.h5ad',
    'tcells.tissue.uc': '../data/tcells.tissue.uc.integrated.h5ad',
    'tcells.pbmc.scps': '../data/tcells.pbmc.scps.integrated.h5ad',
    'tregs.tissue.scps': '../data/tregs.tissue.scps.integrated.annotated.h5ad'
}
for key, file_name in file_names.items():
    adatas[key] = sc.read_h5ad(file_name)

In [3]:
import os
os.environ['R_HOME'] = '/users/daniel.malzl/.conda/envs/scpython/lib/R'
os.environ['TZ'] = 'Europe/Vienna' # needs to be set in some cases to avoid tzlocal error

In [4]:
# needed to copy this from milo due to some weird problem with pandas
# that only seemed to appear when executing milopy.utils.annotate_nhoods all of a sudden
import scipy


def annotate_nhoods(adata, anno_col):
    try:
        nhood_adata = adata.uns["nhood_adata"]
    except KeyError:
        raise KeyError(
            'Cannot find "nhood_adata" slot in adata.uns -- please run milopy.make_nhoods_adata(adata)'
        )

    # Check value is not numeric
    if pd.api.types.is_numeric_dtype(adata.obs[anno_col]):
        raise ValueError(
            'adata.obs[anno_col] is not of categorical type - please use milopy.utils.annotate_nhoods_continuous for continuous variables')

    anno_dummies = pd.get_dummies(adata.obs[anno_col])
    anno_count = adata.obsm["nhoods"].T.dot(
        scipy.sparse.csr_matrix(anno_dummies.values))
    try:
        anno_frac = (anno_count/anno_count.sum(1)).toarray()
    except AttributeError: # for old version of python
        anno_frac = np.array(anno_count/anno_count.sum(1))

    anno_frac = pd.DataFrame(anno_frac,
                             columns=anno_dummies.columns,
                             index=adata.uns["nhood_adata"].obs_names
                             )
    adata.uns["nhood_adata"].obsm["frac_annotation"] = anno_frac.values
    # Turn this to list so that writing out h5ad works
    adata.uns["nhood_adata"].uns["annotation_labels"] = anno_frac.columns.to_list()
    adata.uns["nhood_adata"].uns["annotation_obs"] = anno_col
    adata.uns["nhood_adata"].obs["nhood_annotation"] = anno_frac.idxmax(1)
    adata.uns["nhood_adata"].obs["nhood_annotation_frac"] = anno_frac.max(1)

In [5]:
import milopy
import milopy.core as milo

control_conditions = {
    'tcells.tissue.scps': 'normal', 
    'tcells.tissue.ad': 'normal', 
    'tcells.tissue.uc': 'control', 
    'tcells.pbmc.scps': 'normal',
    'tregs.tissue.scps': 'normal'
}
nhood_annotation_columns = {
    'tcells.tissue.scps': 'condition', 
    'tcells.tissue.ad': 'condition', 
    'tcells.tissue.uc': 'condition', 
    'tcells.pbmc.scps': 'condition',
    'tregs.tissue.scps': 'sat1_status_majority_vote'
}
for k, adata in adatas.items():
    adata.obs['condition'] = 'healthy'
    adata.obs.loc[adata.obs.status != control_conditions[k], 'condition'] = 'disease'
    # needs to be done in order to enforce disease vs healthy comparison
    # otherwise will take alphabetical order and do healthy vs disease
    adata.obs.loc[:, 'condition'] = pd.Categorical(
        adata.obs.condition,
        categories = ['healthy', 'disease']
    )
    milo.make_nhoods(adata)
    milo.count_nhoods(adata, sample_col = 'sample_id')
    milo.DA_nhoods(adata, design="~condition")
    
    anno_col = nhood_annotation_columns[k]
    annotate_nhoods(adata, anno_col = anno_col)

/tmp/ipykernel_10637/458100557.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  adata.obs.loc[:, 'condition'] = pd.Categorical(
/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/users/daniel.malzl/.conda/envs/scpython/lib/python3.9/site-packages/rpy2/robjects/vectors.py:1017: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
/users/daniel.malzl/.conda/envs/scpython/lib/python3.9/site-packages/milopy/core.py:285: FutureWarning: In a future version of pandas all arguments of concat except for the argumen

In [6]:
for i, (k, adata) in enumerate(adatas.items()):
    anno_col = nhood_annotation_columns[k]
    fig, axs = plt.subplots(1, 4)
    sc.pl.umap(
        adata,
        color = 'FOXP3',
        frameon = False,
        show = False,
        size = 50,
        ax = axs[0],
        vmax = 5
    )
    
    sc.pl.umap(
        adata,
        color = 'SAT1',
        frameon = False,
        show = False,
        size = 50,
        ax = axs[1],
        vmax = 20
    )
    
    milopy.utils.build_nhood_graph(adata)
    milopy.plot.plot_nhood_graph(
        adata, 
        alpha=0.1, 
        min_size=5, 
        ax = axs[2], 
        show = False
    )
    
    plot.evaluate.plot_nhood_violin(
        adata,
        0.25,
        ax = axs[-1]
    )
    
    for ax in axs:
        ax.set_title(ax.get_title(), fontsize = 20)
        
    for ax in fig.axes[-2:]:
        ax.tick_params(
            labelsize = 15
        )
    fig.set_figheight(5)
    fig.set_figwidth(20)
    fig.tight_layout()
    fig.savefig(f'../plots/{k}.milo.da.png')

/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/groups/pavri/bioinfo/daniel/Neuwirth_Malzl_et_al_2024/notebooks/sctools/plot/evaluate.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(
/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/groups/pavri/bioinfo/daniel/Neuwirth_Malzl_et_al_2024/notebooks/sctools/plot/evaluate.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(
/users/daniel.malzl/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarn

In [ ]:
for k, adata in adatas.items():
    adata.write(
        f'../data/{k}.integrated.milo.h5ad'
    )